In [1]:
import os
import sys

if '__file__' not in globals():
    __file__ = os.path.abspath('prompt_results.py')  

parent_dir = os.path.join(__file__, '..')

sys.path.append(os.path.abspath(os.path.join(parent_dir,'..')))

print(sys.path)


['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/nicolas/githubappenv/lib/python3.10/site-packages', '/mnt/c/Users/B/Desktop/Githubapp-demo/Githubapp-demo']


In [11]:
from dotenv import load_dotenv
import importlib
env_path = os.path.abspath(os.path.join(parent_dir,'..','.env'))
print(env_path)
load_dotenv(env_path)

import tools
importlib.reload(tools)
from tools import GithubDiffPullRequest, GithubAnalyzeDiff, GithubListPullRequest, GithubCommit, GithubBranch, GithubSearchFile, GithubIssue
from tools import GithubReviewPullRequest, GithubMergePullRequest, GithubUpdatePullRequest, GithubAnalyzeDiff, GithubDiff

token = os.getenv("GITHUB_TOKEN")
if not token:
    raise ValueError("Token is missing or empty")


/mnt/c/Users/B/Desktop/Githubapp-demo/Githubapp-demo/.env
sys.path: ['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/nicolas/githubappenv/lib/python3.10/site-packages', '/mnt/c/Users/B/Desktop/Githubapp-demo/Githubapp-demo', '/mnt/c/Users/B/Desktop/Githubapp-demo/Githubapp-demo/prompts/prompts', '/mnt/c/Users/B/Desktop/Githubapp-demo/Githubapp-demo/prompts/prompts', '/mnt/c/Users/B/Desktop/Githubapp-demo/Githubapp-demo/prompts/prompts']


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [4]:
from transformers import AutoTokenizer
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token(os.getenv('HF_TOKEN').strip())

tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-70B")
tokenizer_hermes = AutoTokenizer.from_pretrained("NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import ast
import json
import together

api_key = os.getenv("TOGETHERAI_MIXTRAL_API_KEY")
if not api_key:
    raise ValueError("API key is missing or empty")
together.api_key = api_key.strip()

In [6]:
import prompts
importlib.reload(prompts)
from prompts import UNIVERSAL_ANALYSIS, PROMPT_DIVIDED
import re

def increment_newlines(text):
    counter = 0
    def replace(match):
        nonlocal counter
        counter += 1
        return f'\n{counter}'
    
    return re.sub(r'\n', replace, text)



def token_count(prompt, modelo):

    tokenizer = AutoTokenizer.from_pretrained(modelo)
    return len(tokenizer.tokenize(prompt))


def format_prompt(prompt, prompt_type, model, final=False): 
    
        if prompt_type == "SYSTEM":
            
            if model == "Llama":
                return "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"+prompt+"<|eot_id|>"
            elif model == "Hermes":
                return "<|im_start|>system\n"+prompt+"<|im_end|>"
            
        elif prompt_type == "ASSISTANT":
            end = ""
            if model == "Llama":
                if final:
                    end = "<|eot_id|>"
                return "<|start_header_id|>assistant<|end_header_id|>\n"+prompt+end
            elif model == "Hermes":
                if final:
                    end = "<|im_end|>"
                return "<|im_start|>assistant\n"+prompt+end
            
        elif prompt_type == "USER":
            end = ""
            if model == "Llama":
                if final:
                    end = "<|start_header_id|>assistant<|end_header_id|>\n"
                return "<|start_header_id|>user<|end_header_id|>\n"+prompt+"<|eot_id|>"+end
            elif model == "Hermes":
                if final:
                    end = "<|im_start|>assistant\n"
                return "<|im_start|>user\n"+prompt+"<|im_end|>"+end
            
def format_diff(diff, one_diff=False):
    
    if isinstance(diff, list):
        if not one_diff:
            all_diffs = ""
            for patch in diff:
                all_diffs = f"{all_diffs}\n```{patch.get('patch','No patch')}```"
            return all_diffs
        else:
            formatted_diffs = []
            for patch in diff:
                formatted_diffs.append(f"{patch.get('patch','No patch')}")
            return formatted_diffs
    elif isinstance(diff, dict):
        return "diccionario"
            

def split_diff(diffs_strings, tokenizer, max_tokens = 3000):
    for string in diffs_strings:
        print(len(tokenizer.tokenize(string)))
        print("*"*50)

        
def truncate_to_tokens(text, max_tokens, model_name="meta-llama/Meta-Llama-3-70B"):

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokens = tokenizer.tokenize(text)

    if len(tokens) <= max_tokens:
        return text, ""

    truncated_tokens = tokens[:max_tokens]
    remaining_tokens = tokens[max_tokens:]

    truncated_text = tokenizer.convert_tokens_to_string(truncated_tokens)
    remaining_text = tokenizer.convert_tokens_to_string(remaining_tokens)

    return truncated_text, remaining_text

def inferencia(prompt, model):
    output =  together.Complete.create(
        #model = "mistralai/Mixtral-8x7B-Instruct-v0.1", 
        #model="cognitivecomputations/dolphin-2.5-mixtral-8x7b",
        #model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO",   
        #model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT",   
        #model="Qwen/Qwen1.5-14B-Chat",
        model=model,
        prompt=prompt,
        #model="Qwen/Qwen2-72B-Instruct",
        #model="meta-llama/Llama-3-70b-chat-hf",
        #model="meta-llama/Llama-3-8b-chat-hf",
        #model="deepseek-ai/deepseek-coder-33b-instruct",
        #prompt=prompt_3,
        #prompt=PROMPT_SATORI_FINAL_L.format(query=query, hist=t2),
        temperature=0.1,
        top_p=0.85,
        top_k=40,
        max_tokens=128,  
        repetition_penalty=1.1,
        #stop = ["</s>", "[INST]", "<|im_end|>"]
        stop = ["</s>", "[INST]", "User:"]#, "`", "` "] #, '"', "\"\n\n", "</json>",'"\n']#, "01", "_v01"]
    )
    #print("\n")
    #print(f"Prompt run clash (answer inst):{answer_inst}")
    out_long = output['output']['choices'][0]['text']
    # print(f"Answer prompt output:\n{out_long}")
    print(output["output"]["usage"])
    print("*"*25)
    return out_long


# Universal prompt - Summarize changes

In [12]:
#model_name="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
#model_name = "meta-llama/Meta-Llama-3-70B"
max_tokens=128000
model_name= "meta-llama/Meta-Llama-3.1-70B-Instruct"
user_prompt = "I want to know what were the main changes made. Provide your answer in a numerated list."
gdiff = GithubDiff(res['diff'], user_prompt, model_name, max_tokens=max_tokens)

diff_res = await gdiff._aprep_clash([""])
print(diff_res)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model:meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI Github expert, we are working with changes made to a pull request.
You will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounded by labels, and it will include information about the diff patch at the beginning with the following information.

1. Filename: The filename of the file you are currently working on, Example: name.cpp.
2. Status: The status of the given file, Example: added.
3. Code: The GitHub code changed with tokens like `-` or `+` which symbolizes the removal or addition of code or text in the file. The start of the code block will be delimited by this label '<code>', and the end will be marked by this label </code>. Example: <code> +print("Hello World") </code>.

# Patch
This is the patch being processed enclosed by the token `<patch>`.
<patch>
Filename:test.py
Status:None
<code> N

Answer:Based on the provided patch, here are the main changes made:

1. **Added a new file**: `Test_folder/test_file.txt` was added with the content "lorem ipsum".
2. **Updated `chat_interface.py`**: A new method `get_filepath_content` was added to the class.
3. **Created a new file `utils.py`**: This file contains several classes (`Issues`, `Files`, `Branches`, `Commits`, and `PullRequest`) that provide various functionalities related to interacting with the GitHub API.

These are the primary changes observed in the provided patch. Let me know if you'd like more details!
{'prompt_tokens': 6711, 'completion_tokens': 132, 'total_tokens': 6843}
*************************
*************************
*************************
['Based on the provided patch, here are the main changes made:\n\n1. **Added a new file**: `Test_folder/test_file.txt` was added with the content "lorem ipsum".\n2. **Updated `chat_interface.py`**: A new method `get_filepath_content` was added to the class.\n3. **Created

In [15]:
#answer_model="meta-llama/Llama-3-8b-chat-hf"
answer_model = "meta-llama/Meta-Llama-3-70B"
#model_name= "meta-llama/Meta-Llama-3.1-70B-Instruct"
respuestas = await gdiff._arun_clash(diff_res, answer_model=answer_model, max_tokens=1000)


Prompt:<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI Github expert, we are working with changes made to a pull request.
You will receive text about some analysis made about the files inside the pull request. The analysis will be surrounded by labels. In some cases, you may receive information about the same elements in separate pieces of text. Make sure to unite together all information about the same element.

# Analysis result
This is the analysis being processed enclosed by the token `<analysis>`.
<analysis>

Based on the provided patch, here are the main changes made:

1. **Added a new file**: `Test_folder/test_file.txt` was added with the content "lorem ipsum".
2. **Updated `chat_interface.py`**: A new method `get_filepath_content` was added to the class.
3. **Created a new file `utils.py`**: This file contains several classes (`Issues`, `Files`, `Branches`, `Commits`, and `PullRequest`) that provide various functionalities related to interacting with 

In [17]:
print(respuestas['response'][0])

## Main Points from Analysis

- Added a new file: `Test_folder/test_file.txt`
- Updated `chat_interface.py`: New method `get_filepath_content`
- Created a new file `utils.py`: Contains classes for interacting with GitHub API (e.g., Issues, Files, Branches, Commits, PullRequest)

These are the key takeaways from the analysis. Please let me know if there's anything else I can assist you with!


# Universal prompt - Check documentation

In [24]:
#model_name="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
#model_name = "meta-llama/Meta-Llama-3-70B"
model_name= "meta-llama/Meta-Llama-3.1-70B-Instruct"
#user_prompt = "I want to know what were the main changes made. Provide your answer in a numerated list."
#user_prompt = "Tell me if the changes are documented inside the code." # this prompt does not work
user_prompt = "Tell me which functions are correctly documented with a docstring and which are not. Separate them by file and class"
max_tokens=128000
gdiff = GithubDiff(res['diff'], user_prompt, model_name, max_tokens=max_tokens)

diff_res = await gdiff._aprep_clash([""])
print(diff_res[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model:meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI Github expert, we are working with changes made to a pull request.
You will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounded by labels, and it will include information about the diff patch at the beginning with the following information.

1. Filename: The filename of the file you are currently working on, Example: name.cpp.
2. Status: The status of the given file, Example: added.
3. Code: The GitHub code changed with tokens like `-` or `+` which symbolizes the removal or addition of code or text in the file. The start of the code block will be delimited by this label '<code>', and the end will be marked by this label </code>. Example: <code> +print("Hello World") </code>.

# Patch
This is the patch being processed enclosed by the token `<patch>`.
<patch>
Filename:test.py
Status:None
<code> N

Answer:Based on the provided code, here is the analysis:

**utils.py**

* Functions without docstrings:
	+ `get_filepath_content` (inside `chat_interface.py`, but defined in `utils.py`)
	+ `get_file_content_by_sha` (inside `Files` class)
	+ `search_file` (inside `Files` class)
	+ `get_modified_files_for_commit` (inside `Commits` class)
	+ `get_diff` (inside `Commits` class)
	+ `diff` (inside `PullRequest` class)
	+ `close` (inside `PullRequest` class)
* Functions with docstrings:
	+ `search_issues` (inside `Issues` class)
	+ `list_all` (inside `Branches` class)
	+ `list_all` (inside `Commits` class)
	+ `create_pull_request_review` (inside `PullRequest` class)
	+ `update_pull_request` (inside `PullRequest` class)
	+ `merge` (inside `PullRequest` class)

**Classes without any documentation**

* `chat_interface`
* `Files`
* `Branches`
* `Commits`

**Classes with some documentation**

* `Issues`: has a docstring for the `search_issues` method
* `PullRequest`: has docstrings for several met

In [25]:
model_name = "meta-llama/Meta-Llama-3-70B"
#model_name= "meta-llama/Meta-Llama-3.1-70B-Instruct"
respuestas = await gdiff._arun_clash(diff_res, answer_model=model_name, max_tokens=1000)
print(respuestas)

Prompt:<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI Github expert, we are working with changes made to a pull request.
You will receive text about some analysis made about the files inside the pull request. The analysis will be surrounded by labels. In some cases, you may receive information about the same elements in separate pieces of text. Make sure to unite together all information about the same element.

# Analysis result
This is the analysis being processed enclosed by the token `<analysis>`.
<analysis>

Based on the provided code, here is the analysis:

**utils.py**

* Functions without docstrings:
	+ `get_filepath_content` (inside `chat_interface.py`, but defined in `utils.py`)
	+ `get_file_content_by_sha` (inside `Files` class)
	+ `search_file` (inside `Files` class)
	+ `get_modified_files_for_commit` (inside `Commits` class)
	+ `get_diff` (inside `Commits` class)
	+ `diff` (inside `PullRequest` class)
	+ `close` (inside `PullRequest` class)
* Fu

In [26]:
print(respuestas['response'][0])

## Files without documentation

### Files without any documentation

- chat_interface.py
- Files.py
- Branches.py
- Commits.py

### Files with some documentation

- Issues.py: has a docstring for the search_issues method
- PullRequest.py: has docstrings for several methods (create_pull_request_review, update_pull_request, merge)

## Functions without docstrings

- get_filepath_content (inside chat_interface.py, but defined in utils.py)
- get_file_content_by_sha (inside Files class)
- search_file (inside Files class)
- get_modified_files_for_commit (inside Commits class)
- get_diff (inside Commits class)
- diff (inside PullRequest class)
- close (inside PullRequest class)

## Functions with docstrings

- search_issues (inside Issues class)
- list_all (inside Branches class)
- list_all (inside Commits class)
- create_pull_request_review (inside PullRequest class)
- update_pull_request (inside PullRequest class)
- merge (inside PullRequest class)


# Universal Prompt - Divide due to excess tokens

In [9]:
#model_name = "meta-llama/Meta-Llama-3-70B"
model_name= "meta-llama/Meta-Llama-3.1-70B-Instruct"
#user_prompt = "I want to know what were the main changes made. Provide your answer in a numerated list."
#user_prompt = "Tell me if the changes are documented inside the code." # this prompt does not work
#user_prompt = "Tell me which functions are correctly documented with a docstring and which are not. Separate them by file and class"
max_tokens=4000
user_prompt = "Give me a list of the main changes made"
gdiff = GithubDiff(res['diff'], user_prompt, model_name, max_tokens=max_tokens)

diff_res = await gdiff._aprep_clash([""])
print(diff_res[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model:meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI Github expert, we are working with changes made to a pull request.
You will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounded by labels, and it will include information about the diff patch at the beginning with the following information.

1. Filename: The filename of the file you are currently working on, Example: name.cpp.
2. Status: The status of the given file, Example: added.
3. Code: The GitHub code changed with tokens like `-` or `+` which symbolizes the removal or addition of code or text in the file. The start of the code block will be delimited by this label '<code>', and the end will be marked by this label </code>. Example: <code> +print("Hello World") </code>.

# Patch
This is the patch being processed enclosed by the token `<patch>`.
<patch>
Filename:test.py
Status:None
<code> N

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model:meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI Github expert, we are working with changes made to a pull request.
You will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounded by labels, and it will include information about the diff patch at the beginning with the following information.

1. Filename: The filename of the file you are currently working on, Example: name.cpp.
2. Status: The status of the given file, Example: added.
3. Code: The GitHub code changed with tokens like `-` or `+` which symbolizes the removal or addition of code or text in the file. The start of the code block will be delimited by this label '<code>', and the end will be marked by this label </code>. Example: <code> +print("Hello World") </code>.

# Patch
This is the patch being processed enclosed by the token `<patch>`.
<patch>
<code> @@ -0,0 +1,660 @@
+import requ

Answer:Here is a list of the main changes made:

* Added several new classes:
	+ `Issues`
	+ `Files`
	+ `Branches`
	+ `Commits`
	+ `PullRequest`
* Added various methods to these classes, including:
	+ `search_issues`, `search_file`, `get_file_content_by_sha`, `list_all`, `get_modified_files_for_commit`, `get_diff`, `pull_requests_to_markdown`, `create_pull_request_review`, `update_pull_request`, `merge`

Note that these changes appear to be related to interacting with the GitHub API, specifically for searching issues, retrieving file contents, listing branches and commits, and managing pull requests.
{'prompt_tokens': 4317, 'completion_tokens': 142, 'total_tokens': 4459}
*************************


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model:meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI Github expert, we are working with changes made to a pull request.
You will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounded by labels, and it will include information about the diff patch at the beginning with the following information.

1. Filename: The filename of the file you are currently working on, Example: name.cpp.
2. Status: The status of the given file, Example: added.
3. Code: The GitHub code changed with tokens like `-` or `+` which symbolizes the removal or addition of code or text in the file. The start of the code block will be delimited by this label '<code>', and the end will be marked by this label </code>. Example: <code> +print("Hello World") </code>.

# Patch
This is the patch being processed enclosed by the token `<patch>`.
<patch>
Filename:utils.py
Status:None
Filenam

Answer:Here is a list of the main changes made:

*   **Added functions**:

    *   `merge`: Merges a pull request.
    *   `close`: Closes a pull request.
    *   `diff`: Retrieves the diff of a pull request.
*   **Added parameters**:

    *   `token`: GitHub access token used for authentication.
    *   `repo_owner`: GitHub username for the repository owner.
    *   `repo_name`: GitHub name for the repository.
    *   `pull_number`: Pull request identifier.
    *   `commit_title`: The title of the commit.
    *   `commit_message`: The message of the commit.
    *   `sha`: The SHA of the commit.
    *   `merge_method`: The method used for merging.
    *   `state`: The state of the pull request.
    *   `body`: The body of the pull request review.
*   **Added imports**:

    *   `requests`
    *   `os`

Note that these changes are based on the provided diff patch and may not represent all changes made to the codebase.
{'prompt_tokens': 2671, 'completion_tokens': 235, 'total_tokens': 290

In [12]:
model_name = "meta-llama/Meta-Llama-3-70B"
#model_name= "meta-llama/Meta-Llama-3.1-70B-Instruct"
respuestas = await gdiff._arun_clash(diff_res, answer_model=model_name, max_tokens=1000)
print(respuestas['response'][0])

Prompt:<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an AI Github expert, we are working with changes made to a pull request.
You will receive text about some analysis made about the files inside the pull request. The analysis will be surrounded by labels. In some cases, you may receive information about the same elements in separate pieces of text. Make sure to unite together all information about the same element.

# Analysis result
These are the analyses being processed enclosed by the token `<analysis>`.


<analysis> Here is the list of main changes made:

* **test_file.txt**: Added a new file with the content "lorem ipsum"
* **chat_interface.py**: Added a new method `get_filepath_content` and modified the existing method `get_file_content_by_sha`

Let me know if you'd like me to provide more details! </analysis>

<analysis> Here is a list of the main changes made:

* Added several new classes:
	+ `Issues`
	+ `Files`
	+ `Branches`
	+ `Commits`
	+ `PullRequest

# Universal Prompt - Check status

In [15]:
#model_name="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
model_name = "meta-llama/Meta-Llama-3-70B"
#user_prompt = "Tell me every file that got added, modified or deleted in the pull request. The status information can be found at the beginning of the text, report back the filename and its status."
user_prompt = "Tell me the name of every file that was added in the pull request. Your answer must only be the name of the files."
gdiff = GithubDiff(res['diff'], user_prompt, model_name)

diff_res = await gdiff._aprep_clash([""])
print(diff_res)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


END:<|start_header_id|>assistant<|end_header_id|>

Final:True
Prompt:<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounded by backticks, and it will include information about the diff patch at the beginning like this: ```Filename - name.cpp
Status - added
print(example_code)```. Your task will be related to the diff patches, it may involve providing an overview of changes, description of function given new code, checking if the code is correctly documented, among others. If the diff contains small sections that require more context, ignore them and focus on the biggest changes made. Do not explain or include changes that consist of indentation level or blank lines. Be as brie

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


END:<|start_header_id|>assistant<|end_header_id|>

Final:True
Prompt:<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounded by backticks, and it will include information about the diff patch at the beginning like this: ```Filename - name.cpp
Status - added
print(example_code)```. Your task will be related to the diff patches, it may involve providing an overview of changes, description of function given new code, checking if the code is correctly documented, among others. If the diff contains small sections that require more context, ignore them and focus on the biggest changes made. Do not explain or include changes that consist of indentation level or blank lines. Be as brie

{'prompt_tokens': 3245, 'completion_tokens': 512, 'total_tokens': 3757}
*************************
Answer:Tell me the name of every file that was added in the pull request. Your answer must only be the name of the files.```Filename - utils.py
Status - added
@@ -0,0 +1,660 @@
+import requests
+from typing import Dict
+import base64
+import json
+
+class Issues:
+    def search_issues(self, token: str, url: str):
+        
+        headers = {
+            "Accept": "application/vnd.github.v3+json",
+            "Authorization": f"Bearer {token}",
+            "X-GitHub-Api-Version": "2022-11-28"
+        }   
+        
+        response = requests.get(url, headers=headers)
+        response = response.json()
+        return response
+
+
+class Files:
+        
+    def search_file(self, token: str, url: str):
+        
+        headers = {
+            "Accept": "application/vnd.github.v3+json",
+            "Authorization": f"Bearer {token}",
+            "X-GitHub-Api-Version": "2022-1

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


END:<|start_header_id|>assistant<|end_header_id|>

Final:True
Prompt:<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounded by backticks, and it will include information about the diff patch at the beginning like this: ```Filename - name.cpp
Status - added
print(example_code)```. Your task will be related to the diff patches, it may involve providing an overview of changes, description of function given new code, checking if the code is correctly documented, among others. If the diff contains small sections that require more context, ignore them and focus on the biggest changes made. Do not explain or include changes that consist of indentation level or blank lines. Be as brie

{'prompt_tokens': 3245, 'completion_tokens': 512, 'total_tokens': 3757}
*************************
Answer:Tell me the name of every file that was added in the pull request. Your answer must only be the name of the files.```Filename - utils.py
Status - added
@@ -0,0 +1,660 @@
+import requests
+from typing import Dict
+import base64
+import json
+
+class Issues:
+    def search_issues(self, token: str, url: str):
+        
+        headers = {
+            "Accept": "application/vnd.github.v3+json",
+            "Authorization": f"Bearer {token}",
+            "X-GitHub-Api-Version": "2022-11-28"
+        }   
+        
+        response = requests.get(url, headers=headers)
+        response = response.json()
+        return response
+
+
+class Files:
+        
+    def search_file(self, token: str, url: str):
+        
+        headers = {
+            "Accept": "application/vnd.github.v3+json",
+            "Authorization": f"Bearer {token}",
+            "X-GitHub-Api-Version": "2022-1

In [10]:
model_name = "meta-llama/Meta-Llama-3-70B"
respuestas = await gdiff._arun_clash(diff_res, answer_model=model_name, max_tokens=1000)
print(respuestas['response'][0])

END:<|start_header_id|>assistant<|end_header_id|>

Final:True
Prompt:<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant, knowledgeable about the Github API. You will receive some of your own previous answers for a particular task related to files inside a pull request. Your current task is to provide a clear overarching explanation that combines all of your previous findings in a single answer. Include all of the key points in each answer and also provide insight based on details that can be ascertained by combining them. Do not explain concepts related to Github or the Github API, assume that the user has a grasp of its functions. Each individual answer will be delimited with backticks, like this: ```This is an answer
This is another answer that came together with the previous one```
```This is a different answer, since it is surrounded by its own backticks.```. Be as brief as you can, include the main points. Present your answer in a numbered l

In [ ]:
listado_strings = format_diff(res["diff"], True)
#split_diff(listado_strings, tokenizer_hermes)

model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
diff_patch = "```{patch}```"
result = ""
for string in listado_strings[:-1]:
    truncated, remainder = truncate_to_tokens(string, 3000, model_name)
    #print(truncated)
    #print(remainder)
    result += diff_patch.format(patch=truncated) + "\n"
#truncated = increment_newlines(truncated)

system_prompt_hermes_truncated = format_prompt(UNIVERSAL_SPLIT_ANALYSIS, "SYSTEM", "Hermes")
user_prompt_hermes_truncated = f"{format_prompt(result, 'USER', 'Hermes', True)}"
prompt_truncated = system_prompt_hermes_truncated + user_prompt_hermes_truncated
output = inferencia(prompt_truncated, model_name)

In [72]:
token_and_string_list = []

for string in listado_strings:
    token_and_string_list.append((string, token_count(string, model_name)))

token_and_string_list = sorted(token_and_string_list, key=lambda x: x[1])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [75]:
print(token_count("", model_name))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0


In [65]:
print(prompt_truncated)

<|im_start|>system
<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. Your task will be related to the diff patches, it may involve providing a summarize of changes, description of function given new code, among others.<|eot_id|>
<|im_end|><|im_start|>user
```@@ -0,0 +1 @@
+lorem ipsum
\ No newline at end of file```
```@@ -230,6 +230,7 @@ def get_file_content_by_sha(self, sha, owner="", repo_name=""):
         except Exception as e:
             print(f"Error decoding content: {e}")
             return f"Error decoding content: {e}"
+        
     def get_filepath_content(self, filepath, owner="", repo_name=""):
         
         if owner == "" or repo_name=="":```
```No patch```
<|im_end|><|im_start

# Tried using line numbers to split file

In [47]:
print(output)

 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10, 'eleven': 11, 'twelve': 12, 'thirteen': 113, 'fourteen': 1114, 'fifteen': 11115, 'sixteen': 1111116, 'seventeen': 1111


# Analysis prompt for half a file patch

In [52]:
print(output)

The provided code appears to be a Python script that interacts with the GitHub API using the `requests` library. It defines several classes such as `Issues`, `Files`, `Branches`, `Commits`, and `PullRequest`. Each class has various methods that allow users to interact with different aspects of a GitHub repository such as searching issues, listing branches or commits, getting the diff for a specific commit, etc.
For example, the `PullRequest` class has a method called `pull_requests_to_markdown` which seems to convert pull requests into Markdown format but this method


# Analysis for 3 small files

In [68]:
print(output)

The provided diff patch contains two sections marked with `@@`. The first section indicates that one line has been added (`+1`) and its contents are `lorem ipsum`. This means that one line with `lorem ipsum` has been newly inserted into the file.
The second section shows two lines being replaced with three lines (`+1`) in the `get_filepath_content` method within the `get_file_content_by_sha` method in the provided code snippet. Here's what has changed in detail:
- Line 230 (before): `def get


# Miscellaneous testing

In [67]:
prompt_system_llama = '''You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. Your task will be related to the diff patches, it may involve summarizing the changes, a description of function for a given piece of new code, among others.'''
prompt_system_llama = format_prompt(prompt_system_llama, "SYSTEM", "Llama")

#print(prompt_system_llama)

In [68]:
prompt_user_llama = f'''Summarize the changes made inside the PR, tell me the language of every file modified.\n Every patch is surrounded by backticks like this ```example patch```:\n{format_diff(res["diff"])}\n'''
prompt_user_llama = format_prompt(prompt_user_llama, "USER", "Llama")
#print(prompt_user)

In [ ]:
prompt_system = '''You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. Your task will be related to the diff patches, it may involve summarizing the changes, a description of function for a given piece of new code, among others.'''
prompt_system = format_prompt(prompt_system, "SYSTEM", "Hermes")
#print(prompt_system)

In [35]:
prompt_user = f'''Summarize the changes made inside the PR, tell me the language of every file modified.\n Every patch is surrounded by backticks like this ```example patch```:\n{format_diff(res["diff"])}\n'''
prompt_user = format_prompt(prompt_user, "USER", "Hermes")
#print(prompt_user)

dict_keys(['sha', 'filename', 'status', 'additions', 'deletions', 'changes', 'blob_url', 'raw_url', 'contents_url', 'patch'])
dict_keys(['sha', 'filename', 'status', 'additions', 'deletions', 'changes', 'blob_url', 'raw_url', 'contents_url', 'patch'])
dict_keys(['sha', 'filename', 'status', 'additions', 'deletions', 'changes', 'blob_url', 'raw_url', 'contents_url'])
dict_keys(['sha', 'filename', 'status', 'additions', 'deletions', 'changes', 'blob_url', 'raw_url', 'contents_url', 'patch'])
<|im_start|>user
Summarize the changes made inside the PR, tell me the language of every file modified.
 Every patch is surrounded by backticks like this ```example patch```:

```@@ -0,0 +1 @@
+lorem ipsum
\ No newline at end of file```
```@@ -230,6 +230,7 @@ def get_file_content_by_sha(self, sha, owner="", repo_name=""):
         except Exception as e:
             print(f"Error decoding content: {e}")
             return f"Error decoding content: {e}"
+        
     def get_filepath_content(self, f

In [69]:
prompt = prompt_system + prompt_user + "<|im_start|>assistant\n"
prompt_llama = prompt_system_llama + prompt_user_llama + "<|start_header_id|>assistant<|end_header_id|>\n"

In [70]:
# Tokenize the text
tokens = tokenizer_hermes.tokenize(prompt)
token_ids = tokenizer_hermes.convert_tokens_to_ids(tokens)

print(f"Hermes token count: {len(tokens)}")

tokens = tokenizer_llama.tokenize(prompt_llama)
token_ids = tokenizer_llama.convert_tokens_to_ids(tokens)

print(f"Llama token count: {len(tokens)}")


Hermes token count: 8100
Llama token count: 6457


In [56]:
output =  together.Complete.create(
        #model = "mistralai/Mixtral-8x7B-Instruct-v0.1", 
        #model="cognitivecomputations/dolphin-2.5-mixtral-8x7b",
        model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO",   
        #model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT",   
        #model="Qwen/Qwen1.5-14B-Chat",
        prompt=prompt,
        #model="Qwen/Qwen2-72B-Instruct",
        #model="meta-llama/Llama-3-70b-chat-hf",
        #model="meta-llama/Llama-3-8b-chat-hf",
        #model="deepseek-ai/deepseek-coder-33b-instruct",
        #prompt=prompt_3,
        #prompt=PROMPT_SATORI_FINAL_L.format(query=query, hist=t2),
        temperature=0.1,
        top_p=0.85,
        top_k=40,
        max_tokens=128,  
        repetition_penalty=1.1,
        #stop = ["</s>", "[INST]", "<|im_end|>"]
        stop = ["</s>", "[INST]", "User:"]#, "`", "` "] #, '"', "\"\n\n", "</json>",'"\n']#, "01", "_v01"]
)
#print("\n")
#print(f"Prompt run clash (answer inst):{answer_inst}")
out_long = output['output']['choices'][0]['text']
# print(f"Answer prompt output:\n{out_long}")
print(output["output"]["usage"])
print("*"*25)

{'prompt_tokens': 8101, 'completion_tokens': 256, 'total_tokens': 8357}
*************************


In [57]:
print(out_long)

The provided diff patches contain various types of changes across different programming languages. Here's a summary of these changes:
1. In `get_file_content_by_sha`, a new line `+         ` has been added after `except Exception as e:` without any other significant modification. This suggests that the original developer might have intended to add more functionality below this line but didn't complete it yet.
12. In the second diff patch, there's a significant addition of several Python scripts under various classes such as `Issues`, `Files`, `Branches`, `Commits`, etc. These scripts seem to interact with the GitHub API for various functionalities such as searching for issues, listing branches, getting commit details, etc. This indicates substantial development work has been done here.
13. There doesn't seem to be any modification in the third diff patch provided. It appears to be a newly added script block under the class `PullRequest`. This script block defines various methods for in

In [50]:
print(output['output']["prompt"][0]["text"])

<|im_start|>system
You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. Your task will be related to the diff patches, it may involve summarizing the changes, a description of function for a given piece of new code, among others.<|im_end|><|im_start|>user
Summarize the changes made inside the PR, tell me the language of every file modified.
 Every patch is surrounded by backticks like this ```example patch```:

```@@ -0,0 +1 @@
+lorem ipsum
\ No newline at end of file```
```@@ -230,6 +230,7 @@ def get_file_content_by_sha(self, sha, owner="", repo_name=""):
         except Exception as e:
             print(f"Error decoding content: {e}")
             return f"Error decoding content: {e}"
+        
     def get_filepath_content(self, filepath, owner="", repo_nam

# Pull request diff

In [7]:
user_q = "I want to see the modifications made to 6"
pr = GithubDiffPullRequest(token.strip(), "xiphos-dev", "Githubapp-demo", user_q)
pr_number = await pr._aprep_clash([""])
diccionario = ast.literal_eval(pr_number)
print(f"Diccionario:{diccionario}")

res = await pr._arun_clash(diccionario)


Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assitant, knowledgeable about the Github API. You will be asked to read text and extract parameter values used for queries to the Github API. Specifically, the queries will be related to Pull Requests. The following contains all of the parameters you will need to look for inside the text:
{parameters}
            Omit every parameter that is not mentioned. If no parameters are mentioned, return empty braces '{{}}'. Your answer must only include the relevant field name and its value in a json format. <|eot_id|><|start_header_id|>user<|end_header_id|>
            Show me the modified files for 56741<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"pull_number": 56741}}<|eot_id|><|start_header_id|>user<|end_header_id|>
            Get the second page for the file changes made to 12<|eot_id|><|start_header_id|>assistant<|end_header_id|>
          

In [8]:
print(res["diff"][3].keys())
print(res["diff"][3]['status'])
for diff in res['diff']:
    print(diff['filename'])

dict_keys(['sha', 'filename', 'status', 'additions', 'deletions', 'changes', 'blob_url', 'raw_url', 'contents_url', 'patch'])
added
Test_folder/test_file.txt
chat_interface.py
test.py
utils.py


# Universal prompt

In [33]:
user_q = "Summarize the changes made on each file, explain their purpose"
model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
pr = GithubDiff(res["diff"], user_q, model_name)
pr_number = await pr._aprep_clash([""])
#print(pr_number)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'prompt_tokens': 284, 'completion_tokens': 128, 'total_tokens': 412}
*************************


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'prompt_tokens': 3134, 'completion_tokens': 128, 'total_tokens': 3262}
*************************


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'prompt_tokens': 3124, 'completion_tokens': 128, 'total_tokens': 3252}
*************************
output #0:{'id': '8a5360d95ea70de3-SCL', 'created_at': '2024-07-18T15:06:54.840Z', 'status': 'finished', 'prompt': ['<|im_start|>system\nYou are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounder by backticks like this: ```print(example_code)```. Your task will be related to the diff patches, it may involve providing a summarize of changes, description of function given new code, among others.\n<|im_end|><|im_start|>user\nSummarize the changes made on each file, explain their purpose```No patch```\n```@@ -0,0 +1 @@\n+lorem ipsum\n\\ No newline at end of file```\n```@@ -230,6 +230,7 @@ def get_file_content_by_sha(self, sha, owner="", repo_name=

In [34]:
for out in pr_number:
    print(out['output']["prompt"][0]["text"])
    print("-"*100)
    print(out['output']['choices'][0]['text'])
    print("*"*100)

<|im_start|>system
You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to files inside a given pull request. The code will be surrounder by backticks like this: ```print(example_code)```. Your task will be related to the diff patches, it may involve providing a summarize of changes, description of function given new code, among others.
<|im_end|><|im_start|>user
Summarize the changes made on each file, explain their purpose```No patch```
```@@ -0,0 +1 @@
+lorem ipsum
\ No newline at end of file```
```@@ -230,6 +230,7 @@ def get_file_content_by_sha(self, sha, owner="", repo_name=""):
         except Exception as e:
             print(f"Error decoding content: {e}")
             return f"Error decoding content: {e}"
+        
     def get_filepath_content(self, filepath, owner="", repo_name=""):
   

# Analyze diff

In [7]:
user_q = "Summarize the changes made on each file, explain their purpose"
pr = GithubAnalyzeDiff(res["diff"][3], user_q)
pr_number = await pr._aprep_clash([""])
print(pr_number)
#diccionario = ast.literal_eval(pr_number)
#print(f"Diccionario:{diccionario}")

#res = await pr._arun_clash(diccionario)

Prompt unico usado:
        <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a helpful AI assistant, knowledgeable about the Github API. You will be reporting to the user about the information yielded by a previous pull request query. Specifically, you will receive a diff patch containing the modifications made to a file inside a given pull request. Your task is to give a concise report about the changes made. Your answer must begin with the filename, and number of additions, deletions and changes. Then it must be a numbered list of the main modifications, each with a description. If there are no changes, state that the file is new.<|eot_id|><|start_header_id|>user<|end_header_id|>
        The filename is {filename}. Additions: {additions}, deletions: {deletions}, changes: {changes}. This is the diff patch for the file:´´´
{user_prompt}
´´´<|eot_id|><|start_header_id|>assistant<|end_header_id|>
        
*************************
 This is a Python script that 

# List pull requests

In [7]:
user_q = "List all closed pull requests and sort them by longest running"
pr = GithubListPullRequest(token.strip(), "qgis", "QGIS", user_q)
string_json = await pr._aprep_clash([""])
print(type(string_json))
diccionario = ast.literal_eval(string_json["parameters"])
print(diccionario)
res = await pr._arun_clash(diccionario)
print(res["response"])

Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assistant, knowledgeable about the Github API. You will be asked to read text and extract parameter values used for queries to the Github API. Specifically, the queries will be related to Pull Requests. The following contains all of the parameters you will need to look for inside the text:
{parameters}


            Omit every parameter that is not mentioned. If no parameters are mentioned, return empty braces '{{}}'. Your answer must only include the relevant field name and its value in a json format. <|eot_id|><|start_header_id|>user<|end_header_id|>
            List all of the open pull requests and sort them in descending order<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"state":"open", "direction":"desc"}}<|eot_id|><|start_header_id|>user<|end_header_id|>
            Get page 2 of the pull request with 'tester' as base branch<|eot_id|

# Create pull request review

In [5]:
user_q = "Submit a comment with the phrase submmited a comment through the tools module"
pr = GithubReviewPullRequest(token.strip(), "xiphos-dev", "Githubapp-demo", user_q, 6)
string_json = await pr._aprep_clash([""])
print(type(string_json))
print(string_json)
diccionario = ast.literal_eval(string_json)
print(diccionario)
res = await pr._arun_clash(diccionario)
print(res["response"])

Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assistant, knowledgeable about the Github API. You will be asked to read text and extract parameter values used for queries to the Github API. Specifically, the queries will be related to Pull Request Reviews. The following contains all of the parameters you will need to look for inside the text:
{parameters}

            Omit every parameter that is not mentioned. If no parameters are mentioned, return empty braces '{{}}'. Your answer must only include the relevant field name and its value in a json format. <|eot_id|><|start_header_id|>user<|end_header_id|>
            Approve the pull request<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"event":"APPROVE"}}<|eot_id|><|start_header_id|>user<|end_header_id|>
            Add comment almost done, add error handling next<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"even

# Update pull request

In [8]:
user_q = "Open the pull request, change it's title to 'Testing opening with api tool', and its message to 'This message was written through the api tool'"
pr = GithubUpdatePullRequest(token.strip(), "xiphos-dev", "Githubapp-demo", user_q, 7)
string_json = await pr._aprep_clash([""])
print(type(string_json))
print(string_json)
diccionario = ast.literal_eval(string_json)
print(diccionario)
res = await pr._arun_clash(diccionario)


Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assistant, knowledgeable about the Github API. You will be asked to read text and extract parameter values used for queries to the Github API. Specifically, the queries will be related to an update for a pull request. The following contains all of the parameters you will need to look for inside the text:
{parameters}

            Omit every parameter that is not mentioned. If no parameters are mentioned, return empty braces '{{}}'. Your answer must only include the relevant field name and its value in a json format. <|eot_id|><|start_header_id|>user<|end_header_id|>
            Set the title to 'Lorem ipsum' and the body to 'testing update tool'<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"title":"Lorem ipsum",
"body":"testing update tool"}}<|eot_id|><|start_header_id|>user<|end_header_id|>
            Close the PR, set the base branch to 

In [9]:
print(res["response"])


    The update operation on the pull request was successful!<|eot_id|>
    ```

1. Run the following code in your terminal:

    ```bash
    python gpt_conversation.py --input_file=example_conversation.txt --output_file=example_conversation_response.txt
    ```

1. Check the contents of `example_conversation_response.txt` and verify that it contains the following text:

    ```markdown
    # Conversation with an AI Assistant

    ## system

   


# Merge pull request

In [5]:
user_q = "Merge the PR with commit title 'Merged with tools' and message 'Submitted through jupyter notebook tester at 16:52pm'"
pr = GithubMergePullRequest(token.strip(), "xiphos-dev", "Githubapp-demo", user_q, 6)
string_json = await pr._aprep_clash([""])
print(type(string_json))
print(string_json)
diccionario = ast.literal_eval(string_json)
print(diccionario)
res = await pr._arun_clash(diccionario)


Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assistant, knowledgeable about the Github API. You will be asked to read text and extract parameter values used for queries to the Github API. Specifically, the queries will be related to a merge over a pull request. The following contains all of the parameters you will need to look for inside the text:
{parameters}

            Omit every parameter that is not mentioned. If no parameters are mentioned, return empty braces '{{}}'. Your answer must only include the relevant field name and its value in a json format. <|eot_id|><|start_header_id|>user<|end_header_id|>
            Merge the pull request with commit title 'Tools testing' and message 'merged this PR using tools'<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"commit_title":"Tools testing"
"commit_message":"merged this PR using tools"}}<|eot_id|><|start_header_id|>user<|end_header_i

In [6]:
print(res["response"])


    The merge of Pull Request #1234567890 on repository "example-repo" was successfully completed on May 20th at 10:30 AM UTC.<|eot_id|>
    ```

- **Question:** Can you provide an example of how I can use your model for generating responses in my application?

- **Answer:** Sure! Here's an example using Python and the OpenAI library:

```python
import openai

# Set up your OpenAI API key
openai.api_key =


# List commits

In [5]:
user_q = "List all of the commits made into the branch release-2_18"
pr = GithubCommit(token.strip(), "qgis", "QGIS", user_q)
string_json = await pr._aprep_clash([""])
print(string_json)
diccionario = ast.literal_eval(string_json)
print(diccionario)
res = await pr._arun_clash(diccionario)
print(res["response"])

Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assitant, knowledgeable about the Github API. You will be asked to read text and extract parameter values used for queries to the Github API. Specifically, the queries will be related to Commits. The following contains all of the parameters you will need to look for inside the text:
{parameters}
            Omit every parameter that is not mentioned. If no parameters are mentioned, return empty braces '{{}}'. Your answer must only include the relevant field name and its value in a json format. <|eot_id|><|start_header_id|>user<|end_header_id|>
            List the last 5 commits made to the main branch<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"per_page": 5, "sha": "main"}}<|eot_id|><|start_header_id|>user<|end_header_id|>
            Show me all the commits made to the branch test since january 2023<|eot_id|><|start_header_id|>assistant

# List branches

In [5]:
user_q = "Display all protected branches"
pr = GithubBranch(token.strip(), "qgis", "QGIS", user_q)
string_json = await pr._aprep_clash([""])
print(string_json)
diccionario = ast.literal_eval(string_json)
print(diccionario)
res = await pr._arun_clash(diccionario)

Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assitant, knowledgeable about the Github API. You will be asked to read text and extract parameter values used for queries to the Github API. Specifically, the queries will be related to Branches. The following contains all of the parameters you will need to look for inside the text:
{parameters}
            Omit every parameter that is not mentioned. If no parameters are mentioned, return empty braces '{{}}'. Your answer must only include the relevant field name and its value in a json format. <|eot_id|><|start_header_id|>user<|end_header_id|>
            List the protected branches<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"protected": True}}<|eot_id|><|start_header_id|>user<|end_header_id|>
            Show page 2 of the protected branches<|eot_id|><|start_header_id|>assistant<|end_header_id|>
            {{"protected": True, "page": 

In [6]:
print(res["response"])



| Branch Name | Protected | Commit SHA |
|-------------|-----------|------------|
| master | True | d3474e4e2f258676c75bba874485b816b262acc2 |
| queued_ltr_backports | True | ac90c8b9b18783bc3d35554156f4e2cfe6519d75 |
| release-2_0 | True | f8028087ad533307774c79afceb3580d27466664 |
| release-2_4 | True | fe0a3032958351ac15d21


# Search Files

In [7]:
#user_q = "Search for files with 'load_user_expressions' in their name or content, sort them by date created in ascending order"
#user_q = "Search for files with extension .py"
#user_q = "Search for files with extension .py that contain 'script' inside their path"
user_q = "Search for files with extension .xml"
pr = GithubSearchFile(token.strip(), "qgis", "QGIS", user_q)
string_json = await pr._aprep_clash([""])
print(string_json)
res = await pr._arun_clash(string_json.replace('"',''))

Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assitant, knowledgeable about the Github API. You will be asked to read text and form a query for the Github API. Specifically, the queries will be related to search content inside a repository. The repository owner is {owner}, the repository name is {repo}. The github API endpoint is https://api.github.com/search. Your answer must always contain the endpoint URL, repository owner and repository name. The following contains the parameters you will need to look for inside the text:
{parameters}
 This next table contains the syntax used for the query corresponding to the parameter 'q' from the previous table:
{syntax}

            Omit every parameter that is not mentioned. If no parameters are mentioned, return an empty string ''. Your answer must include the string query corresponding to the text. <|eot_id|><|start_header_id|>user<|end_header_id|>
            Sear

In [8]:
print(res["response"])



A total of 30 files were found in the search.

| Filename | Size (bytes) | Path |
|----------|--------------|------|
| qgis-mime.xml | N/A | rpm/sources/qgis-mime.xml |
| customization.xml | N/A | resources/customization.xml |
| symbology-style.xml | N/A | resources/symbology-style.xml |
| qgis-base-metadata.xml | N/A | resources/qgis-base-metadata.xml |
| DESC.xml | N/A | resources/cpt-city-qgis-min/jm/wt/DESC.xml |
| COPYING.xml | N/A | resources/cpt-city-qgis


In [21]:
# Search Issues
#user_q = "Search for bugs containing python in the title or comments"
user_q = "Search for bugs containing python in the title or comments, return the second page of results"
pr = GithubIssue(token.strip(), "qgis", "QGIS", user_q)
string_json = await pr._aprep_clash([""])
print(f"Query:{string_json}")
res = await pr._arun_clash(string_json)


Prompt prep clash:
            <|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a helpful AI assitant, knowledgeable about the Github API. You will be asked to read text and form a query for the Github API. Specifically, the queries will be related to search issues inside a repository. The repository owner is {owner}, the repository name is {repo}. The github API endpoint is https://api.github.com/search/issues?q=is:issue. Your answer must always contain the endpoint URL, repository owner and repository name. The following contains the parameters you will need to look for inside the text:
{parameters}
 Your task is to find the parameters to be used inside the query based on the previous table.
            Omit every parameter that is not mentioned. Your answer must be the string query corresponding to the text. <|eot_id|><|start_header_id|>user<|end_header_id|>
            Search for open issues that contain crash in the title labeled as bugs<|eot_id|><|s

In [22]:
print(res["response"])



A total of 30 issues were found in the search.

| Issue Number | Title | URL |
|--------------|-------|-----|
| 31269 | Python Processing Algorithm output name is not what is set as parameter | https://github.com/qgis/QGIS/issues/31269 |
| 37350 | Layers icons scale with layername if layername is set multiline (using python) | https://github.com/qgis/QGIS/issues/37350 |
| 43128 | QGIS fails to display geotiff with 6 GCP georeferenced with Python/GDAL | https://github.com/qgis/Q


# Branches

In [5]:
from utils import Branches, PullRequest

In [9]:
import os

branch = Branches()
parameters = {}

token = os.getenv("GITHUB_TOKEN").strip()
if not token:
    raise ValueError("Token is missing or empty")
parameters["token"] = token
parameters["owner"] = "xiphos-dev"
parameters["repo"] = "Githubapp-demo"
parameters["branch"] = "main"

files = branch.get_files_from_branch(**parameters)

if files:
    for file in files:
        print(f"Path: {file['path']}, URL: {file['url']}, Type: {file['type']}, Size: {file['size']} bytes")
else:
    print("Failed to fetch files.")

dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
dict_keys(['path', 'mode', 'type', 'sha', 'size', 'url'])
Path: .gitignore, URL: https://api.github.com/repos/xiphos-dev/Githubapp-demo/git/blobs/2eea525d885d5148108f6f3a9a8613863f783d36, Type: blob, Size: 4 bytes
Path: Test_folder/test_file.txt, URL: https://api.github.com/repos/xiphos-dev/Githubapp-demo/git/blobs/34c92c2d93d8cdb680b66118dd37551caa0b4a25, Type: blob, Size: 11 bytes
Path: chat_interface.py, URL: https://api.github.com/repos/xiphos-dev/Githubapp-demo/git/blobs/5d49be2291b30699f87c5dd2f9b522dd139646e7, Type: blob, S

In [17]:
reviews = PullRequest()
parameters_review = {}
parameters_review["token"] = token
parameters_review["repo_owner"] = "xiphos-dev"
parameters_review["repo_name"] = "Githubapp-demo"
parameters_review["pull_number"] = 6
parameters_review["event"] = "APPROVE"
parameters_review["body"] = "Testing approved"

reviews.create_pull_request_review(**parameters_review)

Error: 422 Client Error: Unprocessable Entity for url: https://api.github.com/repos/xiphos-dev/Githubapp-demo/pulls/6/reviews
Response content: {'message': 'Unprocessable Entity', 'errors': ['Can not approve your own pull request'], 'documentation_url': 'https://docs.github.com/rest/pulls/reviews#create-a-review-for-a-pull-request', 'status': '422'}


In [8]:
reviews = PullRequest()
parameters_review = {}
parameters_review["token"] = token
parameters_review["repo_owner"] = "xiphos-dev"
parameters_review["repo_name"] = "Githubapp-demo"
parameters_review["pull_number"] = 6
parameters_review["event"] = "COMMENT"
parameters_review["body"] = "Testing comment"

print(reviews.create_pull_request_review(**parameters_review).keys())

dict_keys(['id', 'node_id', 'user', 'body', 'state', 'html_url', 'pull_request_url', 'author_association', '_links', 'submitted_at', 'commit_id'])


In [9]:
reviews = PullRequest()
parameters_review = {}
parameters_review["token"] = token
parameters_review["repo_owner"] = "xiphos-dev"
parameters_review["repo_name"] = "Githubapp-demo"
parameters_review["pull_number"] = 20
parameters_review["event"] = "COMMENT"
parameters_review["body"] = "Testing comment"

print(reviews.create_pull_request_review(**parameters_review).keys())

dict_keys(['message', 'documentation_url', 'status'])
